In [2]:
import numpy as np
import pandas as pd
import itertools

In [3]:
import pandas as pd
df = pd.read_csv('data/similar.csv', sep = ',')
df.set_index('userID/game',inplace=True)
df_jacc = df.mask(df > 0, 1).fillna(0)
display(df)
display(df_jacc)

,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,3.0,3.0,4.0,NaN,4.0,2.0,3.0,NaN
mike0702,3.0,5.0,4.0,3.0,3.0,NaN,NaN,4.0
zephyros,NaN,4.0,NaN,5.0,NaN,NaN,2.0,1.0
dadvador,2.0,NaN,NaN,4.0,NaN,4.0,4.0,5.0


,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
mike0702,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
zephyros,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
dadvador,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


In [4]:
#print(df_jacc.loc['firechicken'])
def compute_similarity(df):
    result = []
    for item in list(itertools.combinations(df.index, 2)):
        a = df.loc[item[0]].values
        b = df.loc[item[1]].values
        intersection = np.sum(a * b)
        union = np.sum(a + b > 0)
        jacc_sim = intersection/union
        cosine_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
        jacc_dist = 1-jacc_sim
        cosine_dist = 1-cosine_sim
        result.append({"user1":item[0],
                       "user2":item[1],
                       'jaccard_similarity':jacc_sim,
                       'cosine_similarity':cosine_sim,
                       "jaccard_distance":jacc_dist,
                       "cosine_distance":cosine_dist})
    df_result = pd.DataFrame(result)

    return df_result
df_result = compute_similarity(df_jacc)
display(df_result)

,user1,user2,jaccard_similarity,cosine_similarity,jaccard_distance,cosine_distance
0,firechicken,mike0702,0.500000,0.666667,0.500000,0.333333
1,firechicken,zephyros,0.250000,0.408248,0.750000,0.591752
2,firechicken,dadvador,0.375000,0.547723,0.625000,0.452277
3,mike0702,zephyros,0.428571,0.612372,0.571429,0.387628
4,mike0702,dadvador,0.375000,0.547723,0.625000,0.452277
5,zephyros,dadvador,0.500000,0.670820,0.500000,0.329180


In [5]:
good_bad = df.mask(df < 3, 0).fillna(0)
good_bad = good_bad.mask(good_bad > 0, 1)
display(df)
display(good_bad)
df_result = compute_similarity(good_bad)
display(df_result)

,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,3.0,3.0,4.0,NaN,4.0,2.0,3.0,NaN
mike0702,3.0,5.0,4.0,3.0,3.0,NaN,NaN,4.0
zephyros,NaN,4.0,NaN,5.0,NaN,NaN,2.0,1.0
dadvador,2.0,NaN,NaN,4.0,NaN,4.0,4.0,5.0


,WatchDog,GTAV,PUBG,StarCraft,CallOfDuty,FIFA,WOW,MineCraft
userID/game,,,,,,,,
firechicken,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
mike0702,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
zephyros,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
dadvador,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


,user1,user2,jaccard_similarity,cosine_similarity,jaccard_distance,cosine_distance
0,firechicken,mike0702,0.571429,0.730297,0.428571,0.269703
1,firechicken,zephyros,0.166667,0.316228,0.833333,0.683772
2,firechicken,dadvador,0.125000,0.223607,0.875000,0.776393
3,mike0702,zephyros,0.333333,0.577350,0.666667,0.422650
4,mike0702,dadvador,0.250000,0.408248,0.750000,0.591752
5,zephyros,dadvador,0.200000,0.353553,0.800000,0.646447


In [6]:
ratings = np.array([[3, 3, 4, 0, 4, 2, 3, 0],
                    [3, 5, 4, 3, 3, 0, 0, 4],
                    [0, 4, 0, 5, 0, 0, 2, 1],
                    [2, 0, 0, 4, 0, 4, 4, 5]])
Ub = (ratings>0).astype(int)
print(Ub)

def jaccard(a,b):
    return (a*b).sum()/((a+b)>0).sum()

users = ["firechicken","mike0702","zephyros","dadvador"]

simmat=np.zeros((4,4))
for i in range(4):
    for j in range(4):
        simmat[i,j] = jaccard(Ub[i],Ub[j])
        if i<j:
            print(users[i]+'-'+users[j], jaccard(Ub[i],Ub[j]))

print(simmat)

[[1 1 1 0 1 1 1 0]
 [1 1 1 1 1 0 0 1]
 [0 1 0 1 0 0 1 1]
 [1 0 0 1 0 1 1 1]]
firechicken-mike0702 0.5
firechicken-zephyros 0.25
firechicken-dadvador 0.375
mike0702-zephyros 0.42857142857142855
mike0702-dadvador 0.375
zephyros-dadvador 0.5
[[1.         0.5        0.25       0.375     ]
 [0.5        1.         0.42857143 0.375     ]
 [0.25       0.42857143 1.         0.5       ]
 [0.375      0.375      0.5        1.        ]]


In [7]:
def cos(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

simmat=np.zeros((4,4))
for i in range(4):
    for j in range(4):
        simmat[i,j] = cos(Ub[i],Ub[j])
        if i<j:
            print(users[i]+'-'+users[j], cos(Ub[i],Ub[j]))
print(simmat)

firechicken-mike0702 0.6666666666666667
firechicken-zephyros 0.4082482904638631
firechicken-dadvador 0.5477225575051661
mike0702-zephyros 0.6123724356957946
mike0702-dadvador 0.5477225575051661
zephyros-dadvador 0.6708203932499369
[[1.         0.66666667 0.40824829 0.54772256]
 [0.66666667 1.         0.61237244 0.54772256]
 [0.40824829 0.61237244 1.         0.67082039]
 [0.54772256 0.54772256 0.67082039 1.        ]]


In [8]:
Ur = (ratings>=3).astype(int)
print(Ur)

simmat=np.zeros((4,4))
for i in range(4):
    for j in range(4):
        simmat[i,j] = jaccard(Ur[i],Ur[j])
        if i<j:
            print(users[i]+'-'+users[j], jaccard(Ur[i],Ur[j]))

[[1 1 1 0 1 0 1 0]
 [1 1 1 1 1 0 0 1]
 [0 1 0 1 0 0 0 0]
 [0 0 0 1 0 1 1 1]]
firechicken-mike0702 0.5714285714285714
firechicken-zephyros 0.16666666666666666
firechicken-dadvador 0.125
mike0702-zephyros 0.3333333333333333
mike0702-dadvador 0.25
zephyros-dadvador 0.2


In [11]:
a = np.array([1,2,3])
b = np.array([4,5,6])

np.dot(a,b)

np.int64(32)

In [12]:
# User A's ratings for movies 1-5
user_ratings = [4, 0, 5, 0, 3]  # 0 means "not rated"

# Similarity between Movie X and movies 1-5
similarities = [0.8, 0.7, 0.9, 0.6, 0.5]

# Naive approach - just take dot product:
naive_prediction = np.dot(user_ratings, similarities)
# = 4*0.8 + 0*0.7 + 5*0.9 + 0*0.6 + 3*0.5
# = 3.2 + 0 + 4.5 + 0 + 1.5 = 9.2

In [ ]:
# Only consider movies the user actually rated
rated_movies = user_ratings > 0  # [True, False, True, False, True]

# Weighted sum using only rated movies
numerator = np.dot(user_ratings, similarities)  # Same as before: 9.2

# Count of valid ratings (using the boolean trick!)
denominator = np.dot(rated_movies.astype(int), similarities)
# = 1*0.8 + 0*0.7 + 1*0.9 + 0*0.6 + 1*0.5
# = 0.8 + 0 + 0.9 + 0 + 0.5 = 2.2

# Normalized prediction
corrected_prediction = numerator / denominator
# = 9.2 / 2.2 ≈ 4.18